In [ ]:
# need to load in rsa and utility infrastructure

In [1]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches 
import seaborn as sns

In [2]:
%load_ext autoreload
%autoreload 2

In [14]:
# Imagined We
sys.path.append('/home/stacyste/Documents/Research/OverloadedSignaling/Algorithms/ImaginedWe')
from mindConstruction import *
from GenerativeSignaler import SignalerZero
from OverloadedReceiver import ReceiverZero
from OverloadedSignaler import SignalerOne
from PragmaticReceiver import ReceiverOne

sys.path.append('/home/stacyste/Documents/Research/OverloadedSignaling/Algorithms')
import constantNames as NC

sys.path.append('/home/stacyste/Documents/Research/OverloadedSignaling/Environments/Experiment')
from consistentSignalChecks_Experiment import SignalIsConsistent_Experiment, SignalIsConsistent_ExperimentSeparatedSignals
from experimentConstruction import *
from setupInference_Experiment import SetupExperiment_SignalsSeparated

In [5]:
#RSA
sys.path.append('/home/stacyste/Documents/Research/OverloadedSignaling/Algorithms/RSA')
from RSAClassical import *
from SetupRSAInference import *
from visualizationRSA import *
from RSAExtensionsForAction import SpeakerActionSignalDistribution

In [6]:
#Utility
sys.path.append('/home/stacyste/Documents/Research/OverloadedSignaling/Algorithms/JointUtility')
from utilityOnlyModel import UtilityDrivenReceiver, UtilityDrivenSignaler, UtilityDrivenSignaler_NoReceiverCosts

# Imagined We

## Example Trial - Irrational Signaler

In [ ]:
rationality = 4
reward = 8
s = (4,0)
r = (4,7)

In [ ]:
runInference = SetupExperiment_SignalsSeparated(rationality, reward)

In [ ]:
signals = ['purple', 'red', 'green', 'triangle']
trgtD = {(3, 6): 'red triangle', (5, 5): 'red circle', (8, 9): 'red square'}
receiverX, signalerX, pragReceiverX = runInference(s, r, signals, trgtD)

In [ ]:
receiverX('purple')

In [ ]:
pragReceiverX('purple')

 Example 2 Costless Reciever

In [6]:
rationality = 4
reward = 8
s = (4,0)
r = (4,7)
signalSpace_GC = ['green', 'circle']
targetDict_GcGtPc = {(0,7): 'green circle', (4,3):'green triangle' , (8,7):'purple circle'}

runInference_costlessReceiver = SetupExperiment_SignalsSeparated(rationality, reward, JointActionUtility_CostlessReceiver)
receiver3, signaler3, pragReceiver3 = runInference_costlessReceiver(s, r, signalSpace_GC, targetDict_GcGtPc)

In [7]:
signaler3({'intentions': 'green circle'})

,probability
signals,
circle,4.999891e-01
green,5.000109e-01
green circle,7.160647e-11


In [8]:
signaler3({'intentions': 'green triangle'})

,probability
signals,
circle,4.299891e-10
green,7.913843e-01
green triangle,2.086157e-01


## Example: Signaler Should Quit

In [ ]:
rationality = 4
reward = 8
s = (4,0)
r = (4,7)
signals = ['purple', 'red', 'green', 'triangle']
trgtD = {(3, 6): 'red triangle', (5, 5): 'red circle', (8, 9): 'red square'}

In [ ]:
runInference = SetupExperiment_SignalsSeparated(rationality, reward)
receiverX, signalerX, pragReceiverX = runInference(s, r, signals, trgtD)

## Step By Step Components

### Setup Params

In [36]:
signalerLocation = (4,0)
receiverLocation = (4,7)
targetDictionary = {(0,2): 'green triangle', (0,3): 'green circle' , (1,3): 'purple circle'}

beta = 4
valueOfReward = 8
signalSpace = ['green', 'circle']

In [37]:
getCost = calculateLocationCost_TaxicabMetric

getUtility = JointActionUtility(costFunction=getCost, 
                                valueOfReward=valueOfReward, 
                                signalerLocation=signalerLocation, 
                                receiverLocation=receiverLocation, 
                                targetDictionary= targetDictionary)

getActionDistribution = ActionDistributionGivenWorldGoal(beta, getUtility)
getMind = GenerateMind(getWorldProbabiltiy_Uniform, getDesireProbability_Uniform, getGoalGivenWorldAndDesire_Uniform, getActionDistribution)


actionSpace = getActionSpace_SignalsSeparated(targetDictionary=targetDictionary,
            signalerPosition = signalerLocation, 
            receiverPosition = receiverLocation)
#condition parameters
condition1 = {'worlds': [1], 
              'desires': [1], 
              'intentions': ['green triangle', 'green circle', 'purple circle'], 
              'actions': actionSpace}
signalCategoryPrior = {'1':1}
signalIsConsistent = SignalIsConsistent_ExperimentSeparatedSignals(targetDictionary, signalerLocation, receiverLocation)

In [38]:
mindPrior = getMind(condition1)
mindPrior.loc[mindPrior['p(mind)']>.001]

p(mind)
intentions     actions          worlds desires          
green circle   ((0, 3), (4, 7)) 1      1        0.327338
               ((4, 0), (0, 3)) 1      1        0.005995
green triangle ((0, 2), (4, 7)) 1      1        0.333331
purple circle  ((1, 3), (4, 7)) 1      1        0.327338
               ((4, 0), (1, 3)) 1      1        0.005995

Utility

In [ ]:
getU = JointActionUtility_CostlessReceiver(costFunction=calculateLocationCost_TaxicabMetric, 
                                    valueOfReward=valueOfReward, 
                                    signalerLocation=signalerLocation, 
                                    receiverLocation=receiverLocation, 
                                    targetDictionary=targetDictionary)

In [ ]:
a = ((5,0), (5,6))
w = 1
g = 'green circle'
getU(a, w, g)

Mind PDFs

In [ ]:
#P(a|w,i)
actionSpace = getActionSpace(targetDictionary, signalDictionary, signalerLocation, receiverLocation, False)
actionPDF = getActionDistribution(actionSpace, 1, 'green triangle')
actionPDF.loc[actionPDF[NC.P_ACTION] > .001]

In [ ]:
#P(i|w,d)
getGoalGivenWorldAndDesire_Uniform (['green triangle', 'green circle', 'purple circle'], 1, 1)

Example P(mind) prior distribution

In [ ]:
#condition parameters
condition2 = {'worlds': [1], 
              'desires': [1], 
              'intentions': ['green circle'], 
              'actions': actionSpace}
mind2 = getMind(condition2)

In [ ]:
mind2.loc[mind2['p(mind)'] > .001]

Signaler Zero - Generative

In [ ]:
generativeSignaler2 = getGenerativeSignaler(targetMind=mind2, signalerCategory=1)
generativeSignaler2.loc[generativeSignaler2['p(signal|mind,c)'] > .01]

In [ ]:
#another example
mind1 = getMind(condition1)
generativeSignaler = getGenerativeSignaler(targetMind=mind1, signalerCategory=1)
generativeSignaler.loc[generativeSignaler['p(signal|mind,c)'] > .01]

Receiver Zero

In [ ]:
rec_green = getReceiverZero('green')
rec_green.loc[rec_green['p(mind|signal)'] > .0001]

In [ ]:
rec_green = getReceiverZero('circle')
rec_green.loc[rec_green['p(mind|signal)'] > .0001]

Pragmatic Signaler

In [ ]:
getSignalerOne({'intentions':'green circle'})

In [ ]:
getSignalerOne({'intentions':'green triangle'})

In [ ]:
a = getSignalerOne({'intentions':'purple circle'})
a

# RSA

## Example Trial

In [20]:
rationality = 4
reward = 8
setupRSAExperimentSpeaker = SetupExperiment_RSASpeakerWithActionChoice(rationality=rationality, valueOfReward=reward)
setupRSAExperimentSpeaker_R0 = SetupExperiment_RSASpeakerWithActionChoice(rationality=rationality, valueOfReward=reward, SpeakerWActionPDF = SpeakerActionSignalDistribution_NoReceiverCost)
setupRSAExperimentSpeakerLanguageOnly = SetupExperiment_RSASpeakerInference(rationality=rationality)
setupRSAExperimentReceiver = SetupExperiment_RSAListenerInference(rationality=rationality)

In [16]:
s = (4,0)
r = (4,7)
signalSpace_GC = ['green', 'circle']
targetDict_GcGtPc = {(0,7): 'green circle', (4,3):'green triangle' , (8,7):'purple circle'}

uniformPrior_3Item = {'green triangle': 1.0/3, 'green circle': 1.0/3, 'purple circle': 1.0/3}

In [17]:
rsaSpeaker = setupRSAExperimentSpeaker(targetPrior = uniformPrior_3Item, signalSpace=signalSpace_GC, targetDictionary=targetDict_GcGtPc, signalerLocation=s, receiverLocation=r)
rsaSpeaker('green triangle')

,probability
signals,
circle,2.319522e-16
green,4.269209e-07
green triangle,9.999996e-01


In [21]:
rsaSpeaker = setupRSAExperimentSpeaker_R0(targetPrior = uniformPrior_3Item, signalSpace=signalSpace_GC, targetDictionary=targetDict_GcGtPc, signalerLocation=s, receiverLocation=r)
rsaSpeaker('green triangle')

,probability
signals,
circle,4.299742e-10
green,7.913915e-01
green triangle,2.086085e-01


In [ ]:
rsaSpeaker = setupRSAExperimentSpeaker(targetPrior = uniformPrior_3Item, signalSpace=signalSpace_GC, targetDictionary=targetDict_GcGtPc, signalerLocation=s, receiverLocation=r)
rsaSpeaker('green triangle')

In [12]:
rsaSpeaker = setupRSAExperimentSpeaker(targetPrior = uniformPrior_3Item, signalSpace=signalSpace_GC, targetDictionary=targetDict_GcGtPc, signalerLocation=s, receiverLocation=r)
rsaSpeaker('green circle')

,probability
signals,
circle,0.499682
green,0.499682
green circle,0.000636


## Same Example but no receiver costs

In [29]:
setupRSASpeaker_NoReceiverCosts = SetupExperiment_RSASpeakerWithActionChoice_NoReceiverCost(rationality=rationality, valueOfReward=reward)

In [30]:
rsaSpeaker_noReceiverCosts = setupRSASpeaker_NoReceiverCosts(targetPrior = uniformPrior_3Item, signalSpace=signalSpace_GC, targetDictionary=targetDict_GcGtPc, signalerLocation=s, receiverLocation=r)
rsaSpeaker_noReceiverCosts('green circle')

,probability
signals,
circle,5.000000e-01
green,5.000000e-01
green circle,7.160803e-11


In [31]:
rsaSpeaker_noReceiverCosts('green triangle')

,probability
signals,
circle,4.299742e-10
green,7.913915e-01
green triangle,2.086085e-01


## Example Case: More Vocab than observed features

In [15]:
items_RT_GC = ['red triangle', 'green circle']
prior_RT_GC = {item:.5 for item in items_RT_GC}
signalSpace_RT_GC = ['green', 'red', 'triangle', 'purple']
targetDict_RT_GC = {(2,8): 'red triangle', (4,3): 'green circle'}
s = (4,0)
r = (4,7)

In [16]:
speaker_TooMuchVocab = setupExperimentSpeaker(targetPrior = prior_RT_GC, signalSpace=signalSpace_RT_GC, targetDictionary=targetDict_RT_GC, signalerLocation=s, receiverLocation=r)
speaker_TooMuchVocab('green circle')

,probability
signals,
green,1.798618e-02
purple,1.590464e-06
red,1.243637e-14
triangle,1.243637e-14
green circle,9.820122e-01


In [17]:
speaker_TooMuchVocab_Language = setupExperimentSpeakerLanguageOnly(targetPrior = prior_RT_GC, signalSpace=signalSpace_RT_GC)
speaker_TooMuchVocab_Language('green circle')

,s1(msg|w)
signals,
purple,1.649485e-01
triangle,1.336082e-143
red,1.336082e-143
green,8.350515e-01


In [18]:
receiver_TooMuchVocab = setupExperimentReceiver(targetPrior = prior_RT_GC, signalSpace=signalSpace_RT_GC)
receiver_TooMuchVocab('red')

/home/stacyste/Documents/Research/OverloadedSignaling/Algorithms/ImaginedWe/../../Algorithms/RSA/RSAClassical.py:31: UserWarning: RSA: Normalizing a distribution w/o mass
  warnings.warn('RSA: Normalizing a distribution w/o mass')
/home/stacyste/Documents/Research/OverloadedSignaling/Algorithms/ImaginedWe/../../Algorithms/RSA/RSAClassical.py:31: UserWarning: RSA: Normalizing a distribution w/o mass
  warnings.warn('RSA: Normalizing a distribution w/o mass')


,l1(w|msg)
intentions,
green circle,2.000000e-36
red triangle,1.000000e+00


# Utility Only Model

In [34]:
# Scenario setups
s = (4,0)
r = (4,7)
rationality = 4
reward = 8

signalSpace_PT = ['purple', 'triangle']
targetDict_PcGtPtGc = {(0,7): 'purple circle', (4,3):'green triangle' , (7,6): 'purple triangle', (8,7):'green circle'}

getUtilReceiverSetup4 = UtilityDrivenReceiver(signalerLocation=s, 
                                          receiverLocation=r, 
                                          targetDictionary=targetDict_PcGtPtGc, 
                                          valueOfReward=reward, 
                                          rationality=rationality)

getUtilSignalerSetup4 = UtilityDrivenSignaler(signalSpace=signalSpace_PT, 
                                          signalerLocation=s, 
                                          receiverLocation=r, 
                                          targetDictionary=targetDict_PcGtPtGc, 
                                          valueOfReward=reward, 
                                          rationality=rationality) 



getUtilReceiverSetup4_NoRecCosts = UtilityDrivenReceiver(signalerLocation=s, 
                                          receiverLocation=r, 
                                          targetDictionary=targetDict_PcGtPtGc, 
                                          valueOfReward=reward, 
                                          rationality=rationality, 
                                          utilityFn = JointActionUtility_CostlessReceiver)

getUtilSignalerSetup4_NoRecCosts = UtilityDrivenSignaler_NoReceiverCosts(signalSpace=signalSpace_PT, 
                                          signalerLocation=s, 
                                          receiverLocation=r, 
                                          targetDictionary=targetDict_PcGtPtGc, 
                                          valueOfReward=reward, 
                                          rationality=rationality) 

In [37]:
getUtilReceiverSetup4('purple').round(4)

p(mind)
intentions      worlds desires actions                  
purple circle   1      1       ((0, 7), (4, 7))      0.0
                               ((4, 0), (0, 7))      0.5
                               ((4, 0), (4, 3))      0.0
                               ((4, 0), (7, 6))      0.0
                               ((4, 0), (8, 7))      0.0
                               ((4, 3), (4, 7))      0.0
                               ((7, 6), (4, 7))      0.0
                               ((8, 7), (4, 7))      0.0
purple triangle 1      1       ((0, 7), (4, 7))      0.0
                               ((4, 0), (0, 7))      0.0
                               ((4, 0), (4, 3))      0.0
                               ((4, 0), (7, 6))      0.5
                               ((4, 0), (8, 7))      0.0
                               ((4, 3), (4, 7))      0.0
                               ((7, 6), (4, 7))      0.0
                               ((8, 7), (4, 7))      0.0

In [38]:
getUtilReceiverSetup4('triangle').round(4)

p(mind)
intentions      worlds desires actions                  
green triangle  1      1       ((0, 7), (4, 7))   0.0000
                               ((4, 0), (0, 7))   0.0000
                               ((4, 0), (4, 3))   0.0177
                               ((4, 0), (7, 6))   0.0000
                               ((4, 0), (8, 7))   0.0000
                               ((4, 3), (4, 7))   0.0000
                               ((7, 6), (4, 7))   0.0000
                               ((8, 7), (4, 7))   0.0000
purple triangle 1      1       ((0, 7), (4, 7))   0.0000
                               ((4, 0), (0, 7))   0.0000
                               ((4, 0), (4, 3))   0.0000
                               ((4, 0), (7, 6))   0.9823
                               ((4, 0), (8, 7))   0.0000
                               ((4, 3), (4, 7))   0.0000
                               ((7, 6), (4, 7))   0.0000
                               ((8, 7), (4, 7))   0.0000

In [35]:
getUtilSignalerSetup4({'intentions':'purple circle'})

,probability
signals,
purple,1.000000e+00
purple circle,6.914400e-13


In [39]:
getUtilSignalerSetup4({'intentions':'purple triangle'})

,probability
signals,
purple,5.000000e-01
triangle,5.000000e-01
purple triangle,2.061154e-09


In [44]:
getUtilSignalerSetup4({'intentions':'green triangle'})

,probability
signals,
triangle,0.017986
green triangle,0.982014


## No Receiver Costs, same setup

In [40]:
getUtilReceiverSetup4_NoRecCosts('purple').round(4)

p(mind)
intentions      worlds desires actions                  
purple circle   1      1       ((0, 7), (4, 7))      0.0
                               ((4, 0), (0, 7))      0.5
                               ((4, 0), (4, 3))      0.0
                               ((4, 0), (7, 6))      0.0
                               ((4, 0), (8, 7))      0.0
                               ((4, 3), (4, 7))      0.0
                               ((7, 6), (4, 7))      0.0
                               ((8, 7), (4, 7))      0.0
purple triangle 1      1       ((0, 7), (4, 7))      0.0
                               ((4, 0), (0, 7))      0.0
                               ((4, 0), (4, 3))      0.0
                               ((4, 0), (7, 6))      0.5
                               ((4, 0), (8, 7))      0.0
                               ((4, 3), (4, 7))      0.0
                               ((7, 6), (4, 7))      0.0
                               ((8, 7), (4, 7))      0.0

In [41]:
getUtilReceiverSetup4_NoRecCosts('triangle').round(4)

p(mind)
intentions      worlds desires actions                  
green triangle  1      1       ((0, 7), (4, 7))      0.0
                               ((4, 0), (0, 7))      0.0
                               ((4, 0), (4, 3))      0.5
                               ((4, 0), (7, 6))      0.0
                               ((4, 0), (8, 7))      0.0
                               ((4, 3), (4, 7))      0.0
                               ((7, 6), (4, 7))      0.0
                               ((8, 7), (4, 7))      0.0
purple triangle 1      1       ((0, 7), (4, 7))      0.0
                               ((4, 0), (0, 7))      0.0
                               ((4, 0), (4, 3))      0.0
                               ((4, 0), (7, 6))      0.5
                               ((4, 0), (8, 7))      0.0
                               ((4, 3), (4, 7))      0.0
                               ((7, 6), (4, 7))      0.0
                               ((8, 7), (4, 7))      0.0

In [42]:
getUtilSignalerSetup4_NoRecCosts({'intentions':'purple circle'})

,probability
signals,
purple,1.000000e+00
purple circle,7.781132e-20


In [43]:
getUtilSignalerSetup4_NoRecCosts({'intentions':'purple triangle'})

,probability
signals,
purple,5.000000e-01
triangle,5.000000e-01
purple triangle,2.319523e-16


In [45]:
getUtilSignalerSetup4_NoRecCosts({'intentions':'green triangle'})

,probability
signals,
triangle,0.999994
green triangle,0.000006
